In [1]:
# Imports essential libraries for data manipulation (pandas), spatial data processing (geopandas), database connection (sqlalchemy), and mapping (leafmap).

import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine,text 
import leafmap

# Set up DB connection parameters within PostGIS

In [2]:
# Define the db connection parameters
username = "postgres"
password = "12345"
host = "localhost"
dbname = "Buildings"
port = "5432"

pg_connection = f"PG:host={host} port={port} dbname={dbname} user={username} password={password}"
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{dbname}")

# Download data from CSDI Portal

In [ ]:
# Download gdb from CSDI Portal

from fgdbDL import download_and_extract_gdb 

url = "https://static.csdi.gov.hk/csdi-webpage/download/51d63757e2675874af80eef94afb6a35/fgdb"

#storage_path = "/home/steeb/Documents/GIS/"
storage_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1"

download_and_extract_gdb(url, storage_path)

In [ ]:
# Download shp from CSDI Portal

from shpDL import download_and_extract_shp

url = 'https://static.csdi.gov.hk/csdi-webpage/download/0e55c533715b5da3ae0ca6e6024e90b4/shp'

storage_path = "/home/steeb/Documents/GIS/"
#storage_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1"

download_and_extract_shp(url, storage_path)

In [ ]:
# Compile OZP data from CSDI Portal using WFS

from ozp2pgsql import fetch_and_process_wfs_data

wfs_url = 'https://www.ozp.tpb.gov.hk/arcgis/services/DATA/OZP_PLAN_CSDI/MapServer/WFSServer?request=GetCapabilities&service=WFS'
download_dir = r"C:\Users\Steve_Lau\Downloads\ozp"
postgis_conn_string = f"postgresql://{username}:{password}@{host}:{port}/{dbname}"

fetch_and_process_wfs_data(wfs_url, download_dir, postgis_conn_string)

# Set up paths and layers

In [3]:
# Define the paths and layer name (comment out either one gdb_path when not in use)

# Building Footprint database
#blg_gdb_path = "/home/steeb/Documents/GIS/20240509/Building_Footprint.gdb"
blg_gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\Building_Footprint.gdb"

# Lot database
#lot_gdb_path = "/home/steeb/Documents/GIS/LandParcel_Lot.gdb"
lot_gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\LandParcel_Lot.gdb"

# Building information and age records
#bdbiar_shp_path = "/home/steeb/Documents/GIS/BDBIAR.shp"
bdbiar_shp_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\BDBIAR.shp"

# Import into a PostgreSQL database using ogr2ogr

In [ ]:
# Imports Building Footprint GDB into a PostgreSQL database using ogr2ogr
from gdb2pgsql import transfer_gdb_to_postgis

transfer_gdb_to_postgis(blg_gdb_path, pg_connection)

In [ ]:
# Imports Lot GDB into a PostgreSQL database using ogr2ogr
from gdb2pgsql import transfer_gdb_to_postgis

transfer_gdb_to_postgis(lot_gdb_path, pg_connection)

In [ ]:
# Imports Building information and age records SHP into a PostgreSQL database using ogr2ogr
from shp2pgsql import import_shapefile_to_postgresql

import_shapefile_to_postgresql(bdbiar_shp_path, pg_connection)

In [ ]:
# Replace variable with GDB path to inspect available layers within the specified GDB

from gdbList import list_layers_with_types

list_layers_with_types(lot_gdb_path)

# Read data from PostgreSQL database into dataframes

In [4]:
table_op = "OCCUPATION_PERMIT"
table_op_blgstr = "OP_BUILDING_STRUCTURE"
table_blgstr = "BUILDING_STRUCTURE"
table_blgcat = "CT_BUILDING_CATEGORY"
table_bdbiar = "BDBIAR"
table_gdf_merged_ozp = "gdf_merged_ozp"

sql_op = text(f"SELECT * FROM {table_op}")
sql_op_blgstr = text(f"SELECT * FROM {table_op_blgstr}")
sql_blstr = text(f"SELECT * FROM {table_blgstr}")
sql_blgcat = text(f"SELECT * FROM {table_blgcat}")
sql_bdbiar = text(f"SELECT * FROM {table_bdbiar}")
sql_gdf_merged_ozp = text(f"SELECT * FROM {table_gdf_merged_ozp}")


In [5]:
# Read the tables into DataFrames
df_op = pd.read_sql(sql_op, con=engine.connect())
df_op_blgstr = pd.read_sql(sql_op_blgstr, con=engine.connect())
df_blgcat = pd.read_sql(sql_blgcat, con=engine.connect())

# Read the tables with geometry into DataFrames
gdf_blgstr = gpd.read_postgis(sql_blstr, con=engine.connect(), geom_col="shape") 
gdf_bdbiar = gpd.read_postgis(sql_bdbiar, con=engine.connect(), geom_col="wkb_geometry")
gdf_merged_ozp = gpd.read_postgis(sql_gdf_merged_ozp, con=engine.connect(), geom_col="geometry")

In [6]:
# Define the remapping dictionary
remap_dict = {
    'Agriculture(1)': 'Agriculture', 
    'Commercial / Residential(1)': 'Commercial / Residential', 
    'Commercial / Residential(2)': 'Commercial / Residential', 
    'Commercial / Residential(3)': 'Commercial / Residential', 
    'Commercial / Residential(4)': 'Commercial / Residential', 
    'Commercial(1)': 'Commercial', 
    'Commercial(2)': 'Commercial', 
    'Commercial(3)': 'Commercial', 
    'Commercial(4)': 'Commercial', 
    'Commercial(5)': 'Commercial', 
    'Commercial(6)': 'Commercial', 
    'Commercial(7)': 'Commercial', 
    'Commercial(8)': 'Commercial', 
    'Commercial(9)': 'Commercial',
    'Commercial (3)': 'Commercial',
    'Commercial (4)': 'Commercial',
    'Commercial (1)': 'Commercial',
    'Commercial (2)': 'Commercial',
    'Commercial(11)': 'Commercial',
    'Commercial(10)': 'Commercial',
    'Comprehensive Development Area(1)': 'Comprehensive Development Area', 
    'Comprehensive Development Area(2)': 'Comprehensive Development Area', 
    'Comprehensive Development Area(3)': 'Comprehensive Development Area', 
    'Comprehensive Development Area(4)': 'Comprehensive Development Area', 
    'Comprehensive Development Area(5)': 'Comprehensive Development Area', 
    'Comprehensive Development Area(6)': 'Comprehensive Development Area', 
    'Conservation Area(1)': 'Conservation Area', 
    'Government, Institution or Community(1)': 'Government, Institution or Community', 
    'Government, Institution or Community(10)': 'Government, Institution or Community', 
    'Government, Institution or Community(2)': 'Government, Institution or Community', 
    'Government, Institution or Community(3)': 'Government, Institution or Community', 
    'Government, Institution or Community(4)': 'Government, Institution or Community', 
    'Government, Institution or Community(5)': 'Government, Institution or Community', 
    'Government, Institution or Community(6)': 'Government, Institution or Community', 
    'Government, Institution or Community(7)': 'Government, Institution or Community', 
    'Government, Institution or Community(8)': 'Government, Institution or Community', 
    'Government, Institution or Community(9)': 'Government, Institution or Community',
    'Government, Institution or Community (1)': 'Government, Institution or Community',
    'Government, Institution or Community(13)': 'Government, Institution or Community',
    'Government, Institution or Community(12)': 'Government, Institution or Community',
    'Government, Institution or Community(11)': 'Government, Institution or Community',
    'Green Belt(1)': 'Green Belt', 
    'Green Belt(2)': 'Green Belt', 
    'Industrial(1)': 'Industrial', 
    'Industrial(2)': 'Industrial', 
    'Industrial(3)': 'Industrial', 
    'Open Space(1)': 'Open Space', 
    'Open Space(2)': 'Open Space', 
    'Open Space(3)': 'Open Space', 
    'Other Specified Uses(1)': 'Other Specified Uses', 
    'Other Specified Uses(2)': 'Other Specified Uses', 
    'Other Specified Uses(3)': 'Other Specified Uses', 
    'Other Specified Uses(4)': 'Other Specified Uses', 
    'Other Specified Uses(5)': 'Other Specified Uses', 
    'Other Specified Uses(6)': 'Other Specified Uses', 
    'Residential (Group A)1': 'Residential (Group A)', 
    'Residential (Group A)10': 'Residential (Group A)', 
    'Residential (Group A)11': 'Residential (Group A)', 
    'Residential (Group A)12': 'Residential (Group A)', 
    'Residential (Group A)13': 'Residential (Group A)', 
    'Residential (Group A)14': 'Residential (Group A)', 
    'Residential (Group A)15': 'Residential (Group A)', 
    'Residential (Group A)16': 'Residential (Group A)', 
    'Residential (Group A)17': 'Residential (Group A)', 
    'Residential (Group A)18': 'Residential (Group A)', 
    'Residential (Group A)19': 'Residential (Group A)', 
    'Residential (Group A)2': 'Residential (Group A)', 
    'Residential (Group A)20': 'Residential (Group A)', 
    'Residential (Group A)21': 'Residential (Group A)', 
    'Residential (Group A)22': 'Residential (Group A)', 
    'Residential (Group A)23': 'Residential (Group A)', 
    'Residential (Group A)24': 'Residential (Group A)', 
    'Residential (Group A)25': 'Residential (Group A)', 
    'Residential (Group A)26': 'Residential (Group A)', 
    'Residential (Group A)3': 'Residential (Group A)', 
    'Residential (Group A)4': 'Residential (Group A)', 
    'Residential (Group A)5': 'Residential (Group A)', 
    'Residential (Group A)6': 'Residential (Group A)', 
    'Residential (Group A)7': 'Residential (Group A)', 
    'Residential (Group A)8': 'Residential (Group A)', 
    'Residential (Group A)9': 'Residential (Group A)', 
    'Residential (Group B)1': 'Residential (Group B)', 
    'Residential (Group B)10': 'Residential (Group B)', 
    'Residential (Group B)11': 'Residential (Group B)', 
    'Residential (Group B)12': 'Residential (Group B)', 
    'Residential (Group B)14': 'Residential (Group B)', 
    'Residential (Group B)16': 'Residential (Group B)', 
    'Residential (Group B)17': 'Residential (Group B)', 
    'Residential (Group B)19': 'Residential (Group B)', 
    'Residential (Group B)2': 'Residential (Group B)', 
    'Residential (Group B)3': 'Residential (Group B)', 
    'Residential (Group B)4': 'Residential (Group B)', 
    'Residential (Group B)5': 'Residential (Group B)', 
    'Residential (Group B)6': 'Residential (Group B)', 
    'Residential (Group B)7': 'Residential (Group B)', 
    'Residential (Group B)8': 'Residential (Group B)', 
    'Residential (Group B)20': 'Residential (Group B)',
    'Residential (Group B)15': 'Residential (Group B)',
    'Residential (Group A)27': 'Residential (Group B)',
    'Residential (Group A)28': 'Residential (Group B)',
    'Residential (Group B)13': 'Residential (Group B)',
    'Residential (Group B)9': 'Residential (Group B)',
    'Residential (Group B)18': 'Residential (Group B)',
    'Residential (Group C)1': 'Residential (Group C)', 
    'Residential (Group C)10': 'Residential (Group C)', 
    'Residential (Group C)11': 'Residential (Group C)', 
    'Residential (Group C)12': 'Residential (Group C)', 
    'Residential (Group C)13': 'Residential (Group C)', 
    'Residential (Group C)14': 'Residential (Group C)', 
    'Residential (Group C)15': 'Residential (Group C)', 
    'Residential (Group C)2': 'Residential (Group C)', 
    'Residential (Group C)3': 'Residential (Group C)', 
    'Residential (Group C)4': 'Residential (Group C)', 
    'Residential (Group C)5': 'Residential (Group C)', 
    'Residential (Group C)6': 'Residential (Group C)', 
    'Residential (Group C)7': 'Residential (Group C)', 
    'Residential (Group C)8': 'Residential (Group C)', 
    'Residential (Group C)9': 'Residential (Group C)', 
    'Residential (Group D)1': 'Residential (Group D)', 
    'Residential (Group E)1': 'Residential (Group E)', 
    'Residential (Group E)2': 'Residential (Group E)', 
    'Village Type Development(1)': 'Village Type Development',
    'Recreation(1)': 'Recreation',
    'Site of Special Scientific Interest(1)': 'Site of Special Scientific Interest',
    'Comprehensive Development Area (2)': 'Comprehensive Development Area',
    'Coastal Protection Area(1)': 'Coastal Protection Area'
}

# Apply the remapping to the GeoDataFrame and keep unmapped values unchanged
gdf_merged_ozp['DESC_ENG'] = gdf_merged_ozp['DESC_ENG'].map(remap_dict).fillna(gdf_merged_ozp['DESC_ENG'])

In [7]:
gdf_merged_ozp.to_postgis(table_name, engine, if_exists="replace")

NameError: name 'table_name' is not defined

In [8]:
# Select only the "opno" and "opdate" columns from df_op
df_op_subset = df_op[["opno", "opdate"]]

df_op_subset.opdate = pd.to_datetime(df_op_subset["opdate"], utc=True)

# Merge df_op_blgstr with the subset of df_op on the "opno" column
df_merge_op_blgstr = pd.merge(df_op_blgstr,
                            df_op_subset,
                            on="opno",
                            how="right")

In [9]:
# Select only the "buildingstructureid" and "opdate" columns from df_merge_op_blgstr
df_merge_op_blgstr_subset = df_merge_op_blgstr[["buildingstructureid", "opno", "opdate"]]

# Merge gdf_blgstr with the subset of df_merge_op_blgstr on the "buildingstructureid" column
gdf_merge_blgstr = pd.merge(gdf_blgstr,
                df_merge_op_blgstr_subset,
                on="buildingstructureid", how="left")

In [10]:
# Select only the "buildingstructureid" and "opdate" columns from df_merge_op_blgstr
df_blgcat_subset = df_blgcat[["code",
                              "description",
                              "note"]]

df_blgcat_subset = df_blgcat_subset.rename(columns={"code": "category",
                                 "description": "catdesc",
                                 "note": "catnote"})

gdf_merge_blgstr.category = gdf_merge_blgstr.category.astype("object").astype("int64")

# Merge gdf_blgstr with the subset of df_merge_op_blgstr on the "buildingstructureid" column
gdf_merge_blgstr = pd.merge(gdf_merge_blgstr,
                df_blgcat_subset,
                on="category", how="left")

In [11]:
today = pd.to_datetime('today', utc=True).normalize()

gdf_merge_blgstr["calcdate"] = today

In [12]:
gdf_merge_blgstr['age'] = (gdf_merge_blgstr["calcdate"] - gdf_merge_blgstr["opdate"]) / pd.Timedelta(days=365)

In [13]:
# Keep only relevant columns
gdf_merge_blgstr = gdf_merge_blgstr.loc[:, ("buildingstructureid",
                    "buildingcsuid",
                    "buildingstructuretype",
                    "catdesc",
                    "catnote",
                    "status",
                    "officialbuildingnameen",
                    "officialbuildingnametc",
                    "numabovegroundstoreys",
                    "numbasementstoreys",
                    "topheight",
                    "baseheight",
                    "opno",
                    "opdate",
                    "age",
                    "shape")]

# Filter Building structure and Building age by "Tower" type

In [14]:
gdf_blgstr_tower = gdf_merge_blgstr[gdf_merge_blgstr.buildingstructuretype == "T"]
gdf_bdbiar_tower = gdf_bdbiar[gdf_bdbiar.nsearch4_e == "Tower"]

In [15]:
gdf_bdbiar_tower.to_crs(epsg=2326, inplace=True)

In [16]:
gdf_sjoin_blgstr = gpd.sjoin(gdf_blgstr_tower, gdf_bdbiar_tower, how="left")

In [17]:
gdf_sjoin_blgstr = gpd.sjoin_nearest(gdf_blgstr_tower, gdf_bdbiar_tower, how="left", max_distance=10)

In [18]:
gdf_sjoin_blgstr.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 333263 entries, 0 to 340950
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   buildingstructureid     333263 non-null  int64              
 1   buildingcsuid           333263 non-null  object             
 2   buildingstructuretype   333263 non-null  object             
 3   catdesc                 333263 non-null  object             
 4   catnote                 333263 non-null  object             
 5   status                  333263 non-null  object             
 6   officialbuildingnameen  59955 non-null   object             
 7   officialbuildingnametc  59410 non-null   object             
 8   numabovegroundstoreys   41898 non-null   float64            
 9   numbasementstoreys      3935 non-null    float64            
 10  topheight               177260 non-null  float64            
 11  baseheight             

In [19]:
# Keep only relevant columns
gdf_sjoin_blgstr = gdf_sjoin_blgstr.loc[:, ("buildingstructureid",
                    "buildingcsuid",
                    "buildingstructuretype",
                    "catdesc",
                    "catnote",
                    "status",
                    "officialbuildingnameen",
                    "officialbuildingnametc",
                    "numabovegroundstoreys",
                    "numbasementstoreys",
                    "topheight",
                    "baseheight",
                    "opno",
                    "opdate",
                    "age",
                    "address_e",
                    "address_c",
                    "search1_e",
                    "search1_c",
                    "search2_e",
                    "search2_c",
                    "nsearch2_e",
                    "nsearch2_c",
                    "nsearch3_e",
                    "nsearch3_c",
                    "nsearch4_e",
                    "nsearch4_c",
                    "nsearch5_e",
                    "nsearch5_c",
                    "shape"
                    )]

In [32]:
gdf_sjoin_blgstr.count()

buildingstructureid       333263
buildingcsuid             333263
buildingstructuretype     333263
catdesc                   333263
catnote                   333263
status                    333263
officialbuildingnameen     59955
officialbuildingnametc     59410
numabovegroundstoreys      41898
numbasementstoreys          3935
topheight                 177260
baseheight                     0
opno                       35879
opdate                     35879
age                        35879
address_e                  54938
address_c                  54928
search1_e                  54938
search1_c                  54938
search2_e                  54938
search2_c                  54938
nsearch2_e                 54938
nsearch2_c                 54938
nsearch3_e                 53716
nsearch3_c                 53716
nsearch4_e                 54938
nsearch4_c                 54938
nsearch5_e                 54938
nsearch5_c                 54938
shape                     333263
dtype: int

In [47]:
# Filter records with values in column A but not in column B
only_opdate = gdf_sjoin_blgstr[gdf_sjoin_blgstr['opdate'].notna() & gdf_sjoin_blgstr['nsearch3_e'].isna()]

# Filter records with values in column B but not in column A
only_nsearch3 = gdf_sjoin_blgstr[gdf_sjoin_blgstr['nsearch3_e'].notna() & gdf_sjoin_blgstr['opdate'].isna()]

# Filter records with values in both column A and column B
both_opdate_and_nsearch3 = gdf_sjoin_blgstr[gdf_sjoin_blgstr['opdate'].notna() & gdf_sjoin_blgstr['nsearch3_e'].notna()]

# Count the records
count_only_opdate = len(only_opdate)
count_only_nsearch3 = len(only_nsearch3)
count_both_opdate_and_nsearch3 = len(both_opdate_and_nsearch3)
count_building_age = count_only_opdate + count_only_nsearch3 + count_both_opdate_and_nsearch3

# Count the total records
total_opdate = gdf_sjoin_blgstr.opdate.count()
total_nsearch3 = gdf_sjoin_blgstr.nsearch3_e.count()

print("Count of records with values in opdate but not in nsearch3_e:", count_only_opdate)
print("Count of records with values in nsearch3_e but not in opdate:", count_only_nsearch3)
print("Count of records with values in both opdate and nsearch3_e:", count_both_opdate_and_nsearch3)
print("Count of records with building age values:", count_building_age)
print("Total of opdate records:", total_opdate)
print("Total of nsearch3_e records:", total_nsearch3)

Count of records with values in opdate but not in nsearch3_e: 2552
Count of records with values in nsearch3_e but not in opdate: 20389
Count of records with values in both opdate and nsearch3_e: 33327
Count of records with building age values: 56268
Total of opdate records: 35879
Total of nsearch3_e records: 53716


In [ ]:
gdf_sjoin_blgstr.to_postgis("gdf_sjoin_blgstr_10m", engine, if_exists="replace")

In [41]:
gdf_sjoin_blgstr.nsearch5_e.count()

54938